In [2]:
import copy
import os
import ssl
import sys
import time
from tempfile import TemporaryDirectory
import sys
sys.path.append("../")
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import utils
from torch.optim import lr_scheduler
from torchvision import models
from arten_model import  UNetDNST
import matplotlib.pyplot as plt

In [14]:
device = "cpu"
model = UNetDNST().to(device)
model.load_state_dict(torch.load("./regression_results/_best_model.pth"))
model.eval()

dataloaders = utils.load_and_process_data(batch_size_train=16, batch_size_val_test=16, num_workers=0)
# for i, data in enumerate(dataloaders['val']):
#     inputs = data["image"]
#     inputs = inputs.float()
#     inputs = inputs.unsqueeze(1) 
   
    
#     original_size = inputs.shape[-2:]
#     labels_septal = data["landmarks"]["leaflet_septal"]
#     labels_septal = utils.readjust_keypoints(labels_septal, original_size)
    

#     labels_lateral = data["landmarks"]["leaflet_lateral"]
#     labels_lateral = utils.readjust_keypoints(labels_lateral, original_size)
#     labels = torch.cat((labels_septal, labels_lateral), dim=1).to(
#                         dtype=torch.float32
#                     )
    
#     break



TTS PROPORTIONS: Train 69.44% | Validation 15.74% | Test 14.81%
TTS PROPORTIONS: Train 69.44% | Validation 15.74% | Test 14.81%
TTS PROPORTIONS: Train 69.44% | Validation 15.74% | Test 14.81%
> Original training size 75
TTS PROPORTIONS: Train 69.44% | Validation 15.74% | Test 14.81%
> Original + augmented data size 150


In [15]:
inputs_lst = []
labels_septal_lst = []
labels_lateral_lst = []
outputs_lst_septal = []
outputs_lst_lateral = []

for i, data in enumerate(dataloaders['test']):
    if i == 1:
        break
    inputs = data["image"]
    inputs = inputs.to(device).float()
    inputs = inputs.unsqueeze(1) 
    outputs = model(inputs) # shape: (batch_size, 10, 4)
    print("outputs.shape", outputs.shape)
    
    original_size = inputs.shape[-2:]

    inputs_lst.append(inputs)

    labels_septal = data["landmarks"]["leaflet_septal"]
    labels_septal = utils.readjust_keypoints(labels_septal, original_size)
    labels_septal_lst.append(labels_septal)

    labels_lateral = data["landmarks"]["leaflet_lateral"]
    labels_lateral = utils.readjust_keypoints(labels_lateral, original_size)
    labels_lateral_lst.append(labels_lateral)

    
    
    outputs_septal = outputs[:, :10, :] # shape: (batch_size, 10, 2)
    outputs_lateral = outputs[:, 10:, :] # shape: (batch_size, 10, 2)
    # detach
    
    outputs_septal = outputs_septal.cpu().detach().numpy()
    outputs_lateral = outputs_lateral.cpu().detach().numpy()

    
    outputs_septal = utils.readjust_keypoints(outputs_septal, original_size)
    outputs_lateral = utils.readjust_keypoints(outputs_lateral, original_size)

    outputs_lst_septal.append(outputs_septal)
    outputs_lst_lateral.append(outputs_lateral)


outputs.shape torch.Size([16, 20, 2])


/Users/giuliamonopoli/opt/anaconda3/envs/deepvalve/lib/python3.11/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:3484.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
